In [4]:
import numpy as np 
import pandas as pd

In [5]:
df = pd.read_csv('D:\FashionAI- RS\submission.csv')

In [9]:
len(df.iloc[0][1].split())

C:\Users\Owner\AppData\Local\Temp\ipykernel_7592\3925576894.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  len(df.iloc[0][1].split())


12

In [2]:
# import pickle
# import csv

# def pickle_to_csv(pickle_file, csv_file):
#     # Step 1: Load data from pickle file
#     with open(pickle_file, 'rb') as f:
#         data = pickle.load(f)
    
#     # Step 2: Convert data to CSV format
#     # Assuming the data is a NumPy array
#     headers = [f"feature_{i}" for i in range(data.shape[1])]  # Assuming each column represents a feature
#     with open(csv_file, 'w', newline='') as f:
#         writer = csv.writer(f)
#         writer.writerow(headers)
#         writer.writerows(data)

# # Usage example:
# pickle_to_csv('D:/FashionAI- RS/articles_latent_matrix.pkl', 'articles_latent_matrix.csv')



positive_transactions_df = pd.read_csv('D:/FashionAI- RS/positive_transactions.csv')

articles_latent_matrix_np = pd.read_csv('D:/FashionAI- RS/collaborative_filtering_rs/articles_latent_matrix.csv')
positive_transactions_df.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0924243001 0781758057 0751471001 0309864012 09...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0924243001 0751471001 0918522001 0924243002 04...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0924243001 0914319002 0751471001 0894668003 09...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0924243001 0751471001 0918522001 0924243002 04...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0924243001 0751471001 0918522001 0924243002 04...


In [3]:
positive_transactions_df.shape

(1371980, 2)

In [4]:
articles_latent_matrix_np.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_990,feature_991,feature_992,feature_993,feature_994,feature_995,feature_996,feature_997,feature_998,feature_999
0,-0.523597,-0.908746,0.366298,-0.698213,0.040383,2.169825,0.774345,0.067102,-0.261164,-0.025457,...,0.534467,1.062947,0.395399,0.957966,1.141902,0.137718,-0.943270,-0.142768,-0.051983,-0.435125
1,-0.101637,0.185542,-0.492651,0.024842,0.227164,0.150778,0.578980,0.537266,-0.912290,-0.913828,...,0.460668,-0.407041,-0.410582,-0.211834,0.268991,-0.108093,0.585754,-0.255044,0.456059,-0.387659
2,-0.090420,-0.125948,0.067930,0.466732,-0.365643,0.481056,0.689512,-0.839367,0.447165,-0.210808,...,0.778598,0.112142,-0.057129,0.154778,0.584196,-0.079730,-0.188274,0.118152,-0.339338,-0.095529
3,-0.415906,0.888904,0.849201,0.803405,0.134218,-0.145065,0.286585,-1.468780,-0.099359,0.637025,...,-0.277569,0.740443,0.912773,2.071180,1.758314,0.452126,0.306712,0.924382,-0.237099,0.015992
4,-1.272878,-0.839160,0.241769,0.286933,-0.057294,-0.243922,0.402757,-0.314359,-0.709894,-0.981846,...,-0.330101,-0.189698,0.072118,-0.506425,0.332933,-0.579324,0.050444,-0.415160,0.297512,0.503568


In [10]:
%pip install fastparquet


  Using cached fastparquet-2024.2.0-cp310-cp310-win_amd64.whl (670 kB)
  Using cached cramjam-2.8.3-cp310-none-win_amd64.whl (1.6 MB)


You should consider upgrading via the 'd:\FashionAI- RS\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [11]:
import pandas as pd
import fastparquet

# Read CSV file
# positive_transactions_df = pd.read_csv('positive_transactions_df.csv')

# Convert to Parquet format
positive_transactions_df.to_parquet('positive_transactions.parquet', index=False)


In [14]:
# 12 items needed?

# Input -> userId, productsPurchased

import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.decomposition import TruncatedSVD

def similar_user(userID, user_articles, positive_transactions_parquet, n_components=12):
    try:
        # Read positive_transactions_parquet
        positive_transactions_df = pd.read_parquet(positive_transactions_parquet)
        
        # Concatenate user_articles with userID and convert to DataFrame
        user_data = pd.DataFrame([user_articles], columns=['article_1', 'article_2', 'article_3', 'article_4', 'article_5', 'article_6', 'article_7', 'article_8', 'article_9', 'article_10', 'article_11', 'article_12'])
        user_data['customer_id'] = userID
        
        # Append user_data to positive_transactions_df
        modified_df = pd.concat([positive_transactions_df, user_data], ignore_index=True)
        
        # Generate a user-item interaction matrix (sparse matrix)
        user_item_matrix = modified_df.drop(columns=['customer_id']).apply(pd.to_numeric, errors='coerce').fillna(0)
        
        # Perform dimensionality reduction using Truncated SVD
        svd = TruncatedSVD(n_components=n_components)
        user_item_matrix_reduced = svd.fit_transform(user_item_matrix)
        
        # Compute similarity scores between the target user and all other users
        similarity_scores = cosine_similarity(user_item_matrix_reduced, user_item_matrix_reduced[-1].reshape(1, -1))
        
        # Get indices of top 3 most similar users (excluding the user itself)
        similar_user_indices = similarity_scores.argsort(axis=0)[-4:-1][::-1].flatten()
        
        # Retrieve customer IDs of top 3 most similar users
        similar_users = positive_transactions_df.iloc[similar_user_indices].index.tolist()
        
        return similar_users
    except Exception as e:
        print(f'Error: {e}')
        return []

# Example usage:
userID = '00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657'
user_articles = ['0924243001', '0781758057', '0309864012', '0918522001', '0863646004', '0924243002', '', '', '', '', '', '']
positive_transactions_parquet = 'positive_transactions.parquet'

# Call the function
similar_users = similar_user(userID, user_articles, positive_transactions_parquet)

print(f"Top 3 most similar users for customer {userID} based on their interactions:")
print(similar_users)



# 0924243001 0781758057 0751471001 0309864012 0918522001 0863646004 0924243002 0872537006 0448509014 0562245001 0915529003 0842976005



# 00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,0924243001 0781758057 0751471001 0309864012 0918522001 0863646004 0924243002 0872537006 0448509014 0562245001 0915529003 0842976005





Top 3 most similar users for customer A based on their interactions:
[457319, 457321, 457322]


In [24]:
import pandas as pd



# Split the 'prediction' column into 12 different columns
positive_transactions_df[['article_1', 'article_2', 'article_3', 'article_4', 'article_5', 'article_6', 'article_7', 'article_8', 'article_9', 'article_10', 'article_11', 'article_12']] = positive_transactions_df['prediction'].str.split(' ', expand=True)

# Drop the original 'prediction' column
positive_transactions_df.drop(columns=['prediction'], inplace=True)

# Print the modified DataFrame
print(positive_transactions_df)


                                               customer_id   article_1  \
0        00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...  0924243001   
1        0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...  0924243001   
2        000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...  0924243001   
3        00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...  0924243001   
4        00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...  0924243001   
...                                                    ...         ...   
1371975  ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...  0924243001   
1371976  ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...  0924243001   
1371977  ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...  0924243001   
1371978  ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...  0924243001   
1371979  ffffd9ac14e89946416d80e791d064701994755c3ab686...  0924243001   

          article_2   article_3   article_4   article_5   article_6  \
0        0781758057  0751471001  0309864